# Notebook for training tensorflow object detector API

## Downloads SSDLite Mobiledet model

In [2]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz -P pretrained_models/
!tar xf pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz --directory pretrained_models/

## Dataset

In [1]:
dataset_path = 'vision/data/'

## Edit pipeline config to configure the training

In [2]:
import tensorflow as tf
print("Tensorflow version", tf.__version__)
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tensorflow version 1.15.0
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
pipeline = pipeline_pb2.TrainEvalPipelineConfig()
config_path = '/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config'
# Read config file and converts it to editable pipeline
with tf.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline)

In [7]:

pipeline.train_input_reader.tf_record_input_reader.input_path[:] = [dataset_path + 'train/Hole.tfrecord']
pipeline.train_input_reader.label_map_path = dataset_path + 'train/Hole_label_map.pbtxt'
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[:] = [dataset_path + 'valid/Hole.tfrecord']
pipeline.eval_input_reader[0].label_map_path = dataset_path + 'valid/Hole_label_map.pbtxt'
pipeline.train_config.fine_tune_checkpoint = 'pretrained_model/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt'
pipeline.train_config.batch_size = 32
pipeline.train_config.num_steps = 10000
pipeline.model.ssd.num_classes = 1
# Enable ssdlite, this should already be enabled in the config we downloaded, but this is just to make sure.
pipeline.model.ssd.box_predictor.convolutional_box_predictor.kernel_size = 3
pipeline.model.ssd.box_predictor.convolutional_box_predictor.use_depthwise = True
pipeline.model.ssd.feature_extractor.use_depthwise = True
# Quantization Aware Traning IMPORTANT!!!!
pipeline.graph_rewriter.quantization.delay = 0
pipeline.graph_rewriter.quantization.weight_bits = 8
pipeline.graph_rewriter.quantization.activation_bits = 8

config_text = text_format.MessageToString(pipeline) # Back to text to write
with tf.gfile.Open(config_path, "wb") as f:
    f.write(config_text)

# This is out config after modifying
!cat config_path

cat: config_path: No such file or directory
